# Pixel-wise comparison
clEsperanto brings some pixel-by-pixel comparison functions which are also available with numpy. Let's see how numpy performs in comparison with our OpenCL stuff. When doing similar comparisons with ImageJ, we saw more performance benefits when GPU-accelerating 3D operations compared to 2D operations. https://clij.github.io/clij-benchmarking/benchmarking_operations_jmh

**Note:** benchmarking results vary heavily depending on image size, kernel size, used operations, parameters and used hardware. Use this notebook to adapt it to your use-case scenario and benchmark on your target hardware. If you have different scenarios or use-cases, you are very welcome to submit your notebook as pull-request!

In [1]:
import pyclesperanto_prototype as cle
import numpy as np
import time

# to measure kernel execution duration properly, we need to set this flag. It will slow down exection of workflows a bit though
cle.set_wait_for_kernel_finish(True)

# selet a GPU with the following in the name. This will fallback to any other GPU if none with this name is found
cle.select_device('RTX')

<GeForce RTX 2070 on Platform: NVIDIA CUDA (1 refs)>

## Comparison operations

In [2]:
# test data
test_image1 = np.random.random([100, 512, 512])
test_image2 = np.random.random([100, 512, 512])

In [3]:
# comparie with numpy
result_image = None

for i in range(0, 10):
    start_time = time.time()
    result_image = np.greater_equal(test_image1, test_image2)
    print("Numpy greater_equal duration: " + str(time.time() - start_time))
    

Numpy greater_equal duration: 0.03446388244628906
Numpy greater_equal duration: 0.03286576271057129
Numpy greater_equal duration: 0.03494763374328613
Numpy greater_equal duration: 0.03191685676574707
Numpy greater_equal duration: 0.032909393310546875
Numpy greater_equal duration: 0.0379033088684082
Numpy greater_equal duration: 0.03354334831237793
Numpy greater_equal duration: 0.03195619583129883
Numpy greater_equal duration: 0.033867835998535156
Numpy greater_equal duration: 0.031914472579956055


In [4]:
# multiply with pyclesperanto
result_image = None

test_image1_gpu = cle.push_zyx(test_image1)
test_image2_gpu = cle.push_zyx(test_image2)

for i in range(0, 10):
    start_time = time.time()
    result_image = cle.greater_or_equal(test_image1_gpu, test_image2_gpu, result_image)
    print("clEsperanto greater_or_equal duration: " + str(time.time() - start_time))

clEsperanto greater_or_equal duration: 0.1508791446685791
clEsperanto greater_or_equal duration: 0.0009965896606445312
clEsperanto greater_or_equal duration: 0.001994609832763672
clEsperanto greater_or_equal duration: 0.000997304916381836
clEsperanto greater_or_equal duration: 0.0009980201721191406
clEsperanto greater_or_equal duration: 0.0019943714141845703
clEsperanto greater_or_equal duration: 0.0009970664978027344
clEsperanto greater_or_equal duration: 0.000997304916381836
clEsperanto greater_or_equal duration: 0.0019948482513427734
clEsperanto greater_or_equal duration: 0.0009989738464355469
